### Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sys import argv
import pytz
import datetime
import multiprocessing as mp
from joblib import Parallel, delayed

### Distance

In [2]:
def select_closest(candidates, origin):
    """Return the index of the closest candidate to a given point."""
    return euclidean_distance(candidates, origin).argmin()

def euclidean_distance(a, b):
    """Return the array of distances of two numpy arrays of points."""
    return np.linalg.norm(a - b, axis=1)

def route_distance(cities):
    """Return the cost of traversing a route of cities in a certain order."""
    points = cities[['x', 'y']]
    distances = euclidean_distance(points, np.roll(points, 1, axis=0))
    return np.sum(distances)

### Neuron

In [3]:
def generate_network(size):
    """
    Generate a neuron network of a given size.

    Return a vector of two dimensional points in the interval [0,1].
    """
    return np.random.rand(size, 2)

def get_neighborhood(center, radix, domain):
    """Get the range gaussian of given radix around a center index."""

    # Impose an upper bound on the radix to prevent NaN and blocks
    if radix < 1:
        radix = 1

    # Compute the circular network distance to the center
    deltas = np.absolute(center - np.arange(domain))
    distances = np.minimum(deltas, domain - deltas)

    # Compute Gaussian distribution around the given center
    return np.exp(-(distances*distances) / (2*(radix*radix)))

def get_route(cities, network):
    """Return the route computed by a network."""
    cities['winner'] = cities[['x', 'y']].apply(
        lambda c: select_closest(network, c),
        axis=1, raw=True)

    return cities.sort_values('winner').index

### Utils

In [4]:
def read_tsp(filename):
    """
    Read a file in .tsp format into a pandas DataFrame

    The .tsp files can be found in the TSPLIB project. Currently, the library
    only considers the possibility of a 2D map.
    """
    with open(filename) as f:
        node_coord_start = None
        dimension = None
        lines = f.readlines()

        # Obtain the information about the .tsp
        i = 0
        while not dimension or not node_coord_start:
            line = lines[i]
            if line.startswith('DIMENSION :'):
                dimension = int(line.split()[-1])
            if line.startswith('NODE_COORD_SECTION'):
                node_coord_start = i
            i = i+1

        #print('Problem with {} cities read.'.format(dimension))

        f.seek(0)

        # Read a data frame out of the file descriptor
        cities = pd.read_csv(
            f,
            skiprows=node_coord_start + 1,
            sep=' ',
            names=['city', 'y', 'x'],
            dtype={'city': str, 'x': np.float64, 'y': np.float64},
            header=None,
            nrows=dimension
        )

        # cities.set_index('city', inplace=True)

        return cities

def normalize(points):
    """
    Return the normalized version of a given vector of points.

    For a given array of n-dimensions, normalize each dimension by removing the
    initial offset and normalizing the points in a proportional interval: [0,1]
    on y, maintining the original ratio on x.
    """
    ratio = (points.x.max() - points.x.min()) / (points.y.max() - points.y.min()), 1
    ratio = np.array(ratio) / max(ratio)
    norm = points.apply(lambda c: (c - c.min()) / (c.max() - c.min()))
    return norm.apply(lambda p: ratio * p, axis=1)

### Plots

In [5]:
def plot_network(cities, neurons, name='diagram.png', ax=None):
    """Plot a graphical representation of the problem"""
    mpl.rcParams['agg.path.chunksize'] = 10000

    if not ax:
        fig = plt.figure(figsize=(5, 5), frameon = False)
        axis = fig.add_axes([0,0,1,1])

        axis.set_aspect('equal', adjustable='datalim')
        plt.axis('off')

        axis.scatter(cities['x'], cities['y'], color='red', s=4)
        axis.plot(neurons[:,0], neurons[:,1], ls='-', color='#0063ba', markersize=2)

        plt.savefig(name, bbox_inches='tight', pad_inches=0, dpi=200)
        plt.close()

    else:
        ax.scatter(cities['x'], cities['y'], color='red', s=4)
        ax.plot(neurons[:,0], neurons[:,1], ls='-', color='#0063ba', markersize=2)
        return ax

def plot_route(cities, route, name='diagram.png', ax=None):
    """Plot a graphical representation of the route obtained"""
    mpl.rcParams['agg.path.chunksize'] = 10000

    if not ax:
        fig = plt.figure(figsize=(5, 5), frameon = False)
        axis = fig.add_axes([0,0,1,1])

        axis.set_aspect('equal', adjustable='datalim')
        plt.axis('off')

        axis.scatter(cities['x'], cities['y'], color='red', s=4)
        route = cities.reindex(route)
        route.loc[route.shape[0]] = route.iloc[0]
        axis.plot(route['x'], route['y'], color='purple', linewidth=1)

        plt.savefig(name, bbox_inches='tight', pad_inches=0, dpi=200)
        plt.close()

    else:
        ax.scatter(cities['x'], cities['y'], color='red', s=4)
        route = cities.reindex(route)
        route.loc[route.shape[0]] = route.iloc[0]
        ax.plot(route['x'], route['y'], color='purple', linewidth=1)
        return ax

### SOM Algorithms

In [6]:
def som_random(problem, iterations, learning_rate=0.8):
    """Solve the TSP using a Self-Organizing Map."""

    # Obtain the normalized set of cities (w/ coord in [0,1])
    cities = problem.copy()
    
    cities[['x', 'y']] = normalize(cities[['x', 'y']])

    # The population size is 8 times the number of cities
    n = cities.shape[0] * 6

    # Generate an adequate network of neurons:
    network = generate_network(n)
    #print('Network of {} neurons created. Starting the iterations:'.format(n))
    
    for i in range(iterations):
        if not i % 100:
            #pass
            print('\t> Iteration {}/{}'.format(i, iterations), end="\r")
                
        # Choose a random city
        city = cities.sample(1)[['x', 'y']].values
        #city = find_further(cities)

        winner_idx = select_closest(network, city)


        # Generate a filter that applies changes to the winner's gaussian
        gaussian = get_neighborhood(winner_idx, n//10, network.shape[0])
        # Update the network's weights (closer to the city)
        network += gaussian[:,np.newaxis] * learning_rate * (city - network)
        # Decay the variables
        learning_rate = learning_rate * 0.99997
        n = n * 0.9997

        path = '../'

        # Check for plotting interval
        if not i % 1000:
            plot_network(cities, network, name=path+'diagrams/{:05d}.png'.format(i))

        # Check if any parameter has completely decayed.
        if n < 1:
            print('Radius has completely decayed, finishing execution',
            'at {} iterations'.format(i))
            break
        if learning_rate < 0.001:
            print('Learning rate has completely decayed, finishing execution',
            'at {} iterations'.format(i))
            break
    else:
        #pass
        print('Completed {} iterations.'.format(iterations))

    plot_network(cities, network, name=path+'diagrams/final.png')

    route = get_route(cities, network)
    #print('cities:',cities)
    #print('route:', route)
    
    plot_route(cities, route, path+'diagrams/route.png')
    return route

#### Defining n

In [17]:
n=40

In [16]:
def kdd(k):
    df = pd.DataFrame()
    
    for i in range(50*k,50*(k+1)):
        print(i)
        #chamada da função{
        problem = read_tsp(f"../data/input{i+1}.tsp")
        route = som_random(problem, 1000000)

        problem = problem.reindex(route)

        distance = route_distance(problem)
                
        #criação do vetor de adjacências
        d = pd.DataFrame(np.zeros((1, 40001)))

        for j in range(len(route)-1):
            d[route[j]*200+route[j+1]+1] = 1
            d[route[j+1]*200+route[j]+1] = 1
        d[route[-1]*200+route[0]+1] = 1
        d[route[0]*200+route[-1]+1] = 1

        d[0]=i

        df = df.append(d)

    columns = []
    columns.append('graph')
    for i1 in range(200):
        for i2 in range(200):
            columns.append(f'a_{i1},{i2}')    

    df.columns = columns

    df = df.astype('int32')

    #time = datetime.datetime.now(pytz.timezone('Brazil/East')).strftime("%d-%m-%Y_%H:%M")

    df.to_csv( f'../outputs/outputs_13/output_{k}.csv', index=False) 
    print("Finished")

#### Use parallel to use all processors

In [17]:
Parallel(n_jobs=n, verbose=100)(delayed(kdd)(i) for i in range(n))

[Parallel(n_jobs=40)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done   1 tasks      | elapsed: 41.4min
[Parallel(n_jobs=40)]: Done   2 out of  40 | elapsed: 41.7min remaining: 792.3min
[Parallel(n_jobs=40)]: Done   3 out of  40 | elapsed: 41.8min remaining: 516.0min
[Parallel(n_jobs=40)]: Done   4 out of  40 | elapsed: 41.9min remaining: 376.7min
[Parallel(n_jobs=40)]: Done   5 out of  40 | elapsed: 42.0min remaining: 293.9min
[Parallel(n_jobs=40)]: Done   6 out of  40 | elapsed: 42.0min remaining: 238.1min
[Parallel(n_jobs=40)]: Done   7 out of  40 | elapsed: 42.0min remaining: 198.1min
[Parallel(n_jobs=40)]: Done   8 out of  40 | elapsed: 42.0min remaining: 168.1min
[Parallel(n_jobs=40)]: Done   9 out of  40 | elapsed: 42.1min remaining: 145.0min
[Parallel(n_jobs=40)]: Done  10 out of  40 | elapsed: 42.1min remaining: 126.4min
[Parallel(n_jobs=40)]: Done  11 out of  40 | elapsed: 42.1min remaining: 111.1min
[Parallel(n_jobs=40)]: Done  12 out of  4

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [18]:
mypath = '../outputs/outputs_13/'

df_final = pd.read_csv(f'{mypath}output_0.csv')
print(df_final.shape)

for i in range(1,n):
    print(i)
    df = pd.read_csv(f'{mypath}output_{i}.csv')
    df_final = pd.concat([df_final,df])
    print(df_final.shape)

df_final.to_csv( f'{mypath}output_final.csv', index=False) 

(50, 40001)
1
(100, 40001)
2
(150, 40001)
3
(200, 40001)
4
(250, 40001)
5
(300, 40001)
6
(350, 40001)
7
(400, 40001)
8
(450, 40001)
9
(500, 40001)
10
(550, 40001)
11
(600, 40001)
12
(650, 40001)
13
(700, 40001)
14
(750, 40001)
15
(800, 40001)
16
(850, 40001)
17
(900, 40001)
18
(950, 40001)
19
(1000, 40001)
20
(1050, 40001)
21
(1100, 40001)
22
(1150, 40001)
23
(1200, 40001)
24
(1250, 40001)
25
(1300, 40001)
26
(1350, 40001)
27
(1400, 40001)
28
(1450, 40001)
29
(1500, 40001)
30
(1550, 40001)
31
(1600, 40001)
32
(1650, 40001)
33
(1700, 40001)
34
(1750, 40001)
35
(1800, 40001)
36
(1850, 40001)
37
(1900, 40001)
38
(1950, 40001)
39
(2000, 40001)
